In [183]:
import os
import numpy as np
import pandas as pd

# load data '../../chars.csv'
chars = pd.read_csv('../characters_freq.csv', sep=',', encoding='utf-8')
chars = chars[chars['level'] == '一级']
chars.head()

words = pd.read_csv('../vocabulary_freq.csv', sep=',', encoding='utf-8')
# words = words[words['level'] == '一级']
words.head()

,index,level,word,pinyin,part_of_speech
0,7.79,一级,的,de,助
1,7.17,一级,是,shì,动
2,7.16,一级,在,zài,动、介、副
3,7.14,一级,了,le,助
4,7.14,三级,了,liǎo,动


In [184]:
# new dataframe
df = pd.DataFrame(columns=['page', 'chars', 'words'])

page = 0
for index, row in chars.iterrows():

    page += 1

    df.loc[index, 'page'] = page
    df.loc[index, 'chars'] = row['character']

df.head()

,page,chars,words
0,1,的,NaN
3,2,是,NaN
4,3,在,NaN
6,4,了,NaN
7,5,我,NaN


In [185]:
df['old_chars'] = ''
for page, row in df.iterrows():
    cum_chars = df.loc[:page, 'chars'].str.cat(sep=', ')
    df.at[page, 'old_chars'] = cum_chars
df

,page,chars,words,old_chars
0,1,的,NaN,的
3,2,是,NaN,"的, 是"
4,3,在,NaN,"的, 是, 在"
6,4,了,NaN,"的, 是, 在, 了"
7,5,我,NaN,"的, 是, 在, 了, 我"
...,...,...,...,...
2386,303,朋,NaN,"的, 是, 在, 了, 我, 和, 有, 不, 人, 你, 也, 他, 这, 中, 年, 对..."
2444,304,汽,NaN,"的, 是, 在, 了, 我, 和, 有, 不, 人, 你, 也, 他, 这, 中, 年, 对..."
2478,305,渴,NaN,"的, 是, 在, 了, 我, 和, 有, 不, 人, 你, 也, 他, 这, 中, 年, 对..."
2542,306,午,NaN,"的, 是, 在, 了, 我, 和, 有, 不, 人, 你, 也, 他, 这, 中, 年, 对..."


In [188]:

def find_words(df, words, num_chars_interval=(2, 10), max_page=-1):
    for page, row in df.iterrows():
        chars = row['old_chars']
        contains = words['word'].apply(lambda x: all(char in chars for char in x) and num_chars_interval[0] <= len(x) <= num_chars_interval[1])

        df.at[page, 'words'] = words[contains]['word'].values.tolist()

        print(f"Page {row['page']}:\n\tChars: {chars}\n\tWords: {(df.at[page, 'words'])}\n")

        # stop at max_page
        if row['page'] == max_page:
            break

find_words(df, words, num_chars_interval=(2, 10), max_page=10)





Page 1:
	Chars: 的
	Words: []

Page 2:
	Chars: 的, 是
	Words: []

Page 3:
	Chars: 的, 是, 在
	Words: []

Page 4:
	Chars: 的, 是, 在, 了
	Words: []

Page 5:
	Chars: 的, 是, 在, 了, 我
	Words: []

Page 6:
	Chars: 的, 是, 在, 了, 我, 和
	Words: []

Page 7:
	Chars: 的, 是, 在, 了, 我, 和, 有
	Words: ['有的', '有的是']

Page 8:
	Chars: 的, 是, 在, 了, 我, 和, 有, 不
	Words: ['有的', '是不是', '有的是']

Page 9:
	Chars: 的, 是, 在, 了, 我, 和, 有, 不, 人
	Words: ['有的', '有人', '是不是', '有的是']

Page 10:
	Chars: 的, 是, 在, 了, 我, 和, 有, 不, 人, 你
	Words: ['有的', '有人', '是不是', '有的是']



In [187]:
df.head(20)

,page,chars,words,old_chars
0,1,的,[],的
3,2,是,[],"的, 是"
4,3,在,[],"的, 是, 在"
6,4,了,[],"的, 是, 在, 了"
7,5,我,[],"的, 是, 在, 了, 我"
8,6,和,[],"的, 是, 在, 了, 我, 和"
11,7,有,"[有的, 有的是]","的, 是, 在, 了, 我, 和, 有"
12,8,不,"[有的, 是不是, 有的是]","的, 是, 在, 了, 我, 和, 有, 不"
13,9,人,"[有的, 有人, 是不是, 有的是]","的, 是, 在, 了, 我, 和, 有, 不, 人"
14,10,你,"[有的, 有人, 是不是, 有的是]","的, 是, 在, 了, 我, 和, 有, 不, 人, 你"
